In [2]:
# Cell 1: Load Data, Sentiment Analysis, and Calculate Daily Returns

import pandas as pd
import numpy as np
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import os

# --- 1. Load News Data and Prepare for Sentiment Analysis (Task 3 Requirement) ---

NEWS_FILE_PATH = '../data/newsData/raw_analyst_ratings.csv'
df_news = pd.read_csv(NEWS_FILE_PATH)

# Robust Date Cleaning (FIXED: Handling mixed timezone awareness)
df_news['date'] = pd.to_datetime(df_news['date'], errors='coerce')
df_news.dropna(subset=['date'], inplace=True)

# Check for timezone-awareness before localizing
if df_news['date'].dt.tz is None:
    # If the column is naive, localize it to the original timezone (UTC-4)
    df_news['date'] = df_news['date'].dt.tz_localize('America/New_York', ambiguous='infer')

# Now that the column is timezone-aware, convert it to the target UTC timezone
df_news['date'] = df_news['date'].dt.tz_convert('UTC')

# Create a normalized date column (only date, no time) for merging
df_news['Date'] = df_news['date'].dt.normalize().dt.date


# Initialize VADER sentiment analyzer (part of nltk)
try:
    sia = SentimentIntensityAnalyzer()
except LookupError:
    nltk.download('vader_lexicon')
    sia = SentimentIntensityAnalyzer()

def get_vader_sentiment(headline):
    """Calculates the compound sentiment score for a given headline."""
    if pd.isna(headline):
        return 0
    # Use the compound score as the final sentiment metric
    return sia.polarity_scores(headline)['compound']

# Apply sentiment analysis (This is a long-running step!)
df_news['sentiment_score'] = df_news['headline'].apply(get_vader_sentiment)

print("--- News Data Sentiment Analysis Complete ---")
print(f"Sentiment Score calculated for {len(df_news)} headlines.")
print(df_news[['headline', 'sentiment_score', 'Date', 'stock']].head())


# --- 2. Load Processed Stock Data and Calculate Daily Returns (Task 3 Requirement) ---

TICKERS = ['AAPL', 'AMZN', 'GOOG', 'META', 'MSFT', 'NVDA']
STOCK_DATA_DIR = '../data/yfinance_data/'
df_stocks = {}

for ticker in TICKERS:
    processed_file = f'processed_{ticker}_data.csv'
    processed_path = os.path.join(STOCK_DATA_DIR, processed_file)
    
    if os.path.exists(processed_path):
        # Load the processed data which already has TA indicators
        df = pd.read_csv(processed_path, index_col='Date', parse_dates=True)
        
        # Calculate Daily Stock Returns (Percentage Change) [cite: 154]
        df['Daily_Return'] = df['Close'].pct_change() * 100
        
        # Drop the first row (which now contains NaN return)
        df.dropna(subset=['Daily_Return'], inplace=True)
        
        df_stocks[ticker] = df
        print(f"Loaded {ticker} and calculated Daily_Return. Shape: {df.shape}")
    else:
        print(f"Warning: Processed file not found for {ticker} at {processed_path}")

print("--- Stock Data Loading and Daily Returns Calculation Complete ---")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Mer\AppData\Roaming\nltk_data...


--- News Data Sentiment Analysis Complete ---
Sentiment Score calculated for 55987 headlines.
                                            headline  sentiment_score  \
0            Stocks That Hit 52-Week Highs On Friday            0.000   
1         Stocks That Hit 52-Week Highs On Wednesday            0.000   
2                      71 Biggest Movers From Friday            0.000   
3       46 Stocks Moving In Friday's Mid-Day Session            0.000   
4  B of A Securities Maintains Neutral on Agilent...            0.296   

         Date stock  
0  2020-06-05     A  
1  2020-06-03     A  
2  2020-05-26     A  
3  2020-05-22     A  
4  2020-05-22     A  
Loaded AAPL and calculated Daily_Return. Shape: (3740, 11)
Loaded AMZN and calculated Daily_Return. Shape: (3740, 11)
Loaded GOOG and calculated Daily_Return. Shape: (3740, 11)
Loaded META and calculated Daily_Return. Shape: (2889, 11)
Loaded MSFT and calculated Daily_Return. Shape: (3740, 11)
Loaded NVDA and calculated Daily_Return.

In [3]:
# Cell 2: Sentiment Aggregation, Data Merging, and Correlation Analysis

# --- 1. Aggregate News Sentiment by Day and Ticker (Task 3 Requirement) ---

# Group the news articles by the stock ticker and the publication date (normalized)
# Then calculate the mean of the sentiment scores.
df_daily_sentiment = df_news.groupby(['stock', 'Date'])['sentiment_score'].mean().reset_index()
df_daily_sentiment.rename(columns={'sentiment_score': 'Avg_Sentiment'}, inplace=True)

print("--- Daily Sentiment Aggregation Complete ---")
print(df_daily_sentiment.head())


# --- 2. Perform Merging and Correlation Analysis (Task 3 Requirement) ---

TICKERS = ['AAPL', 'AMZN', 'GOOG', 'META', 'MSFT', 'NVDA']
correlation_results = {}

for ticker in TICKERS:
    # 2a. Get the Daily Returns data (from Cell 1)
    df_stock = df_stocks.get(ticker)
    
    # 2b. Filter the aggregated sentiment data for the current ticker
    df_sentiment_ticker = df_daily_sentiment[df_daily_sentiment['stock'] == ticker].copy()
    
    # Ensure the 'Date' column in the sentiment data is ready for merge (convert date object to datetime)
    df_sentiment_ticker['Date'] = pd.to_datetime(df_sentiment_ticker['Date'])
    df_sentiment_ticker.set_index('Date', inplace=True)
    
    if df_stock is not None and not df_sentiment_ticker.empty:
        # 2c. Merge the sentiment with the stock data on the 'Date' index
        # We use an inner join to only keep dates where both news and returns exist
        df_merged = df_stock.merge(
            df_sentiment_ticker[['Avg_Sentiment']], 
            left_index=True, 
            right_index=True, 
            how='inner'
        )
        
        # 2d. Calculate Pearson Correlation Coefficient
        # Calculate the correlation between the averaged sentiment and the daily stock return
        correlation = df_merged['Avg_Sentiment'].corr(df_merged['Daily_Return'])
        
        correlation_results[ticker] = {
            'Correlation_Coefficient': correlation,
            'Data_Points_Merged': len(df_merged)
        }
    else:
        correlation_results[ticker] = {
            'Correlation_Coefficient': np.nan,
            'Data_Points_Merged': 0
        }


# --- 3. Present Final Results ---
df_correlation = pd.DataFrame.from_dict(correlation_results, orient='index')
df_correlation.index.name = 'Ticker'
df_correlation['Correlation_Coefficient'] = df_correlation['Correlation_Coefficient'].round(4)
df_correlation.sort_values(by='Correlation_Coefficient', ascending=False, inplace=True)

print("\n--- Final Correlation Analysis: Sentiment vs. Daily Return ---")
print("Pearson Correlation Coefficient (Avg Daily Sentiment vs. Daily % Return)")
print(df_correlation)

--- Daily Sentiment Aggregation Complete ---
  stock        Date  Avg_Sentiment
0     A  2020-05-22         0.0480
1     A  2020-05-26         0.0000
2     A  2020-06-03         0.0000
3     A  2020-06-05         0.0000
4    AA  2020-05-18         0.8519

--- Final Correlation Analysis: Sentiment vs. Daily Return ---
Pearson Correlation Coefficient (Avg Daily Sentiment vs. Daily % Return)
        Correlation_Coefficient  Data_Points_Merged
Ticker                                             
AAPL                     1.0000                   2
NVDA                     0.5227                   4
GOOG                    -0.2344                   5
AMZN                    -1.0000                   2
META                        NaN                   0
MSFT                        NaN                   0
